In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
import datetime
from astropy import units as u
import scopesim as sim
import scopesim_templates as sim_tp
import astropy.units as u
from scopesim.effects.ter_curves import FilterCurve

In [2]:
src = sim_tp.stellar.clusters.cluster(mass=10000, distance=2000, core_radius=5.0)

imf - sample_imf: Setting maximum allowed mass to 10000
imf - sample_imf: Loop 0 added 1.01e+04 Msun to previous total of 0.00e+00 Msun


In [3]:
cmd = sim.UserCommands(
    use_instrument="MICADO",
    set_modes=["IMG_4mas", "SCAO"],
)
cmd.update(properties={
    "!OBS.filter_name": "J_DREAMS",
    "!OBS.ndit": 1, 
    "!OBS.dit": 3600,

    "!OBS.catg": "SCIENCE",
    "!OBS.tech": "IMAGE",
    "!OBS.type": "OBJECT",
    "!OBS.mjdobs": datetime.datetime(2022, 1, 1, 2, 30, 0),

    "!DET.bin_size": 1,

    # "!INST.filter_file_format": "filters/{}.dat",
})
micado = sim.OpticalTrain(cmd)
    

In [4]:
pixel_scale_micado, plate_scale_micado, pixel_size_micado = 0.004, 0.26666666666, 0.015
pixel_size_micado * plate_scale_micado / pixel_scale_micado

0.999999999975

In [5]:
pixel_scale_dreams, plate_scale_dreams, pixel_size_dreams = 2.48, 206.265, 0.012
pixel_size_dreams * plate_scale_dreams / pixel_scale_dreams

0.9980564516129032

In [6]:
pixel_scale = micado.cmds["!INST.pixel_scale"]
plate_scale = micado.cmds["!INST.plate_scale"]
pixel_size = micado['full_detector_array'].data["pixel_size"][0]
pixel_size, plate_scale, pixel_scale, pixel_size * plate_scale / pixel_scale

(0.012, 41.253, 0.496, 0.9980564516129034)

In [ ]:
micado.observe(src)
hdus = micado.readout()

 FOV effects:  50%|█████████████████████████████████████████████████████████████████████████████████                                                                                 | 1/2 [00:00<00:00,  8.41it/s]


In [ ]:
filter_J_MICADO = FilterCurve(filename="filters/TC_filter_J.dat")
filter_J_DREAMS = FilterCurve(filename="filters/TC_filter_J_DREAMS.dat")

In [ ]:
filter_J_MICADO.plot()

In [ ]:
filter_J_DREAMS.plot()

In [ ]:
len(hdus[0])

In [ ]:
detector_order = [2, 1, 4, 3, 6, 5]
detector_order = [1, 1, 1, 1, 1 ,1]  # To test with 1 detector
plt.figure(figsize=(20, 20))
for plot_number, hdu_number in enumerate(detector_order, 1):
    plt.subplot(3, 2, plot_number)
    plt.imshow(hdus[0][hdu_number].data, origin="lower", norm=LogNorm())

plt.savefig("MICADO_with_DREAMS_detector_array.png")

In [ ]:
hdus[0][0].header

In [ ]:
hdus[0].writeto("my_dithered_cluster.fits", overwrite=True)